In [1]:
'''
   Trying MAB
'''

'\n   Trying MAB\n'

In [2]:
# lets get the data first
import pandas as pd
import sqlalchemy

def get_connection():
    engine = sqlalchemy.create_engine('sqlite:////Users/nikhillondhe/projects/didactic-happiness/instance/tasks.db')
    return engine
    
def load_data():
    ''' Load data from sqlite db '''
    query = '''
         SELECT T.*, CAST(strftime('%H', R.rec_ts) as Integer) as rec_ts,
         CASE
             WHEN 
                 W.id IS NOT NULL 
                 AND cast((julianday(end_ts) - julianday(start_ts)) * 24 * 60 * 60 as Integer) > 60
             THEN 1
             ELSE 0
         END AS was_picked
         FROM task T
         INNER JOIN recommendation R
         ON T.id = R.task_id
         LEFT JOIN
         work_log W
         ON R.id = W.rec_id
    '''
    engine = get_connection()
    df = pd.read_sql(query, engine)
    engine.dispose()
    return df

df = load_data()
df

,id,name,complexity,type,due_date,priority,repeatable,status,rec_ts,was_picked
0,6,nlp scratch: seq 2 seq learn,medium,learning,None,medium,0,pending,16,0
1,7,attention in PyTorch,medium,learning,None,low,0,pending,16,0
2,15,read ml book,medium,learning,None,medium,0,done,16,0
3,8,tidy living room,simple,chores,None,low,0,done,16,0
4,16,RL: gaussian LL,medium,learning,None,medium,0,pending,16,0
...,...,...,...,...,...,...,...,...,...,...
3094,32,happiness: mab,medium,constructive,None,medium,0,in_progress,20,0
3095,22,email responses,simple,constructive,None,high,0,pending,20,0
3096,120,aisuite run,simple,learning,None,low,0,pending,20,0
3097,83,roomba,simple,chores,None,low,1,pending,20,0


In [8]:
df['type'].unique()

array(['learning', 'chores', 'constructive', 'creative'], dtype=object)

In [14]:
import numpy as np

def compute_qvalues(df: pd.DataFrame, col: str, rewards: dict, reward_col: str) -> list:
    values = df[col].unique()
    k = len(values)
    arm_to_idx = {v: k for k,v in enumerate(values)}
    print(arm_to_idx)
    Qvalues = np.zeros(k)
    counts = np.zeros(k)

    for idx, row in df.iterrows():
        arm_name = row[col]
        reward = rewards[arm_name] * row[reward_col]
        arm_idx = arm_to_idx[arm_name]
        counts[arm_idx] += 1
        Qvalues[arm_idx] += (reward - Qvalues[arm_idx]) / counts[arm_idx]
    return Qvalues

print(compute_qvalues(df, 'complexity', {'simple': 1, 'medium': 3, 'hard': 7}, 'was_picked'))
print(compute_qvalues(df, 'priority', {'low': 1, 'medium': 3, 'high': 7}, 'was_picked'))
print(compute_qvalues(df, 'type', {'chores': 1, 'creative': 3, 'learning': 7, 'constructive': 15}, 'was_picked'))

{'medium': 0, 'simple': 1, 'hard': 2}
[0.12806324 0.08574673 0.        ]
{'medium': 0, 'low': 1, 'high': 2}
[0.08396125 0.08361582 0.57198444]
{'learning': 0, 'chores': 1, 'constructive': 2, 'creative': 3}
[0.36653895 0.12583893 0.72687225 0.17578125]


In [15]:
def compute_metrics(q_values):
    mean = np.mean(q_values)
    std_dev = np.std(q_values)
    cv = std_dev / mean if mean > 0 else 0
    entropy = -np.sum((q_values / np.sum(q_values)) * np.log(q_values / np.sum(q_values)))
    gini = np.sum(np.abs(np.subtract.outer(q_values, q_values))) / (2 * len(q_values) * np.sum(q_values))
    top_k_concentration = np.sum(sorted(q_values, reverse=True)[:2]) / np.sum(q_values)  # Top-2 concentration

    return {
        "mean": mean,
        "std_dev": std_dev,
        "cv": cv,
        "entropy": entropy,
        "gini": gini,
        "top_2_concentration": top_k_concentration,
    }

arm_types = {
    'complexity': [0.12806324, 0.08574673, 0],
    'priority': [0.08396125, 0.08361582, 0.57198444],
    'type': [0.36653895, 0.12583893, 0.72687225, 0.17578125]
}

# Evaluate each arm type
results = {arm_type: compute_metrics(q_values) for arm_type, q_values in arm_types.items()}

# Display results
for arm_type, metrics in results.items():
    print(f"{arm_type}: {metrics}")


complexity: {'mean': np.float64(0.07126998999999999), 'std_dev': np.float64(0.05327432377543676), 'cv': np.float64(0.7475000877008229), 'entropy': np.float64(nan), 'gini': np.float64(0.3993054829638362), 'top_2_concentration': np.float64(1.0)}
priority: {'mean': np.float64(0.24652050333333334), 'std_dev': np.float64(0.2301377998553537), 'cv': np.float64(0.9335442559281663), 'entropy': np.float64(0.6921823464455177), 'gini': np.float64(0.44023259133645287), 'top_2_concentration': np.float64(0.8869386536895356)}
type: {'mean': np.float64(0.34875784499999996), 'std_dev': np.float64(0.23606165298820578), 'cv': np.float64(0.6768640659200248), 'entropy': np.float64(1.1688721800791742), 'gini': np.float64(0.35731412364358434), 'top_2_concentration': np.float64(0.7837896807740627)}


/var/folders/63/vdsl0bvd413ff898d5q2jwb80000gq/T/ipykernel_45890/1635742412.py:5: RuntimeWarning: divide by zero encountered in log
  entropy = -np.sum((q_values / np.sum(q_values)) * np.log(q_values / np.sum(q_values)))
/var/folders/63/vdsl0bvd413ff898d5q2jwb80000gq/T/ipykernel_45890/1635742412.py:5: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum((q_values / np.sum(q_values)) * np.log(q_values / np.sum(q_values)))
